# Análisis de GW170817
Un ejemplo para hallar la SNR con una plantilla Post-Newtoniana utilizando la libreria pycbc

In [1]:
import sys
!{sys.executable} -m pip install pycbc ligo-common --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.2/201.2 kB 180.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 113.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 170.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 213.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 366.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 228.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 MB 185.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirem

In [ ]:
import numpy as np
import matplotlib.mlab as mlab
from pycbc.waveform import get_fd_waveform, get_td_waveform
from pycbc.waveform import td_approximants, fd_approximants
from pycbc.detector import Detector
from pycbc import waveform
from pycbc.filter import matched_filter
from scipy import signal
from scipy.signal import butter, filtfilt
from scipy.interpolate import interp1d
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.projections as mprj
import h5py

In [ ]:
# Formato para las graficas
LNWDT=1.3; FNT=18
plt.rcParams['lines.linewidth'] = LNWDT; plt.rcParams['font.size'] = FNT
#plt.rcParams['xtick.direction'] = 'out'; plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['xtick.top'] = 'on'; plt.rcParams['ytick.right'] = 'on'
#plt.rcParams['xtick.minor.visible'] = True; plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['font.family'] = 'Times New Roman'
#plt.rcParams['font.serif'] = 'Times'
rc('mathtext', fontset='stix')

In [ ]:
#### Constantes cgs #####
G = 6.67408*10**-8
c = 2.99792458*10**10
Msun = 1.98855*10**33
tSol = G*Msun/c**3

parsec = 3.08568025e18
d_source = 1.e6*parsec

In [ ]:
# funcion para whitening
def whiten(strain, interp_psd, dt):
    Nt = len(strain)
    freqs = np.fft.rfftfreq(Nt, dt)

    hf = np.fft.rfft(strain)
    norm = 1./np.sqrt(1./(dt*2))
    white_hf = hf / np.sqrt(interp_psd(freqs)) * norm
    white_ht = np.fft.irfft(white_hf, n=Nt)
    return white_ht

In [3]:
# -- Set a GPS time:
#t0 = 1126259462.4    # -- GW150914
t0 = 1187008882.4    # -- GW170817

#-- Choose detector as H1, L1, or V1
detector = 'H1'

In [5]:
import requests, os
%config InlineBackend.figure_format = 'retina'

try:
    from gwpy.timeseries import TimeSeries
except:
    ! pip install -q "gwpy==3.0.4"
    from gwpy.timeseries import TimeSeries

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 44.8 MB/s eta 0:00:00


In [6]:
from gwosc.locate import get_urls
url = get_urls(detector, t0, t0)[-1]

print('Downloading: ' , url)
fn = os.path.basename(url)
with open(fn,'wb') as strainfile:
    straindata = requests.get(url)
    strainfile.write(straindata.content)

Downloading:  https://gwosc.org/eventapi/json/GWTC-1-confident/GW170817/v3/H-H1_GWOSC_4KHZ_R1-1187006835-4096.hdf5


In [9]:
strain = TimeSeries.read(fn,format='hdf5.gwosc')

In [15]:
strain.dt

<Quantity 0.00024414 s>

In [10]:
strain

<TimeSeries([-1.90953777e-19, -1.76176603e-19, -1.82604103e-19,
             ..., -6.70231357e-19, -6.61236726e-19,
             -6.77738035e-19]
            unit=Unit(dimensionless),
            t0=<Quantity 1.18700684e+09 s>,
            dt=<Quantity 0.00024414 s>,
            name='Strain',
            channel=None)>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

detectors = ['H1','L1']

#strain, time, channel_dict = rl.loaddata(fileName)
data_H1 = h5py.File('/content/drive/MyDrive/ColabNotebooks/H-H1_LOSC_CLN_4_V1-1187007040-2048.hdf5', "r")
data_L1 = h5py.File('/content/drive/MyDrive/ColabNotebooks/H-H1_LOSC_CLN_4_V1-1187007040-2048.hdf5', "r")

data_in = [data_H1, data_L1]

T = 32*2
tevent = 1842

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
strain_H1 = data_H1["strain"]["Strain"][()]
dt_H1 = data_H1["strain"]["Strain"].attrs["Xspacing"]

strain_L1 = data_L1["strain"]["Strain"][()]
dt_L1 = data_L1["strain"]["Strain"].attrs["Xspacing"]

In [ ]:
if dt_H1 == dt_L1:
    delta_t = dt_H1
else:
    delta_t = dt_H1
    print ('time arrays sampling is not equal')

In [2]:
mchirp = 1.197
q = 1
nu = q / (1.+q)**2
mtot = mchirp / nu**(3/5.)
m1 = 1.3758
m2 = 1.3758
lambda1 = 300
lambda2 = 300*q
print('q  = {0:.2f}'.format(q))
print('m1 = {0:.4f}, m2 = {1:.4f}'.format(m1, m2))
print('m total = {0:.2f}'.format(mtot))
print('mchirp det = {0:.3f}'.format(mchirp))
print((m1*m2/(m1+m2)**2)**(3/5.)*(m1+m2))

fs = int(1/delta_t)
fmin = 30.
fmax = 1600.

q  = 1.00
m1 = 1.3758, m2 = 1.3758
m total = 2.75
mchirp det = 1.197
1.1977034649828076


NameError: ignored

In [ ]:
hp, hc = get_td_waveform(approximant='SpinTaylorT4', mass1=m1, mass2=m2,
                         f_lower=80, delta_t=1./fs,
                         lambda1=300, lambda2=10,
                         phase_order=-1,
                         amplitude_order=-1)